#### Module imports

In [1]:
import torch
import torchvision
import numpy as np
from numpy import asarray
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch import permute,avg_pool1d
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import os
from pathlib import Path
import pickle

#### Loading files

In [2]:
# load pre processed files
imgs_path = Path('/home/richard/data/Schiefer/preprocessed_224')
assert imgs_path.is_dir()
files = sorted(imgs_path.glob('*.png'))

randomizer = np.random.RandomState(seed=99834)
randomizer.shuffle(files)

assert len(files) == 291 # all files are found
print("First 10 files are: {}".format([x.name for x in files[:10]]))

First 10 files are: ['CFB184_ortho_rgb_0079_preprocessed.png', 'CFB184_ortho_rgb_0030_preprocessed.png', 'CFB184_ortho_rgb_0028_preprocessed.png', 'CFB184_ortho_rgb_0130_preprocessed.png', 'CFB184_ortho_rgb_0100_preprocessed.png', 'CFB184_ortho_rgb_0200_preprocessed.png', 'CFB184_ortho_rgb_0105_preprocessed.png', 'CFB184_ortho_rgb_0081_preprocessed.png', 'CFB184_ortho_rgb_0269_preprocessed.png', 'CFB184_ortho_rgb_0222_preprocessed.png']


#### Loading Images

In [5]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

def alter_image(img_name):
    image = Image.open(img_name)
    image_tensor = normalize(to_tensor(image)).unsqueeze(0)
    image_tensor = image_tensor.reshape(1,3,224,224)
    return image_tensor

def load_images_as_tensors(paths):
    images = [alter_image(image) for image in paths]
    return images

In [6]:
image_tensors = load_images_as_tensors(files)
assert len(image_tensors) == 291

#### Feature Extraction

Load the model

In [7]:
class FeatureExtractor(nn.Module):
  def __init__(self, model):
    super(FeatureExtractor, self).__init__()
		# Extract VGG-16 Feature Layers
    self.features = list(model.features)
    self.features = nn.Sequential(*self.features)
		# Extract VGG-16 Average Pooling Layer
    self.pooling = model.avgpool
		# Convert the image into one-dimensional vector
    self.flatten = nn.Flatten()
		# Extract the first part of fully-connected layer from VGG16
    self.fc = model.classifier[0]
  
  def forward(self, x):
		# It will take the input 'x' until it returns the feature vector called 'out'
    out = self.features(x)
    out = self.pooling(out)
    out = self.flatten(out)
    out = self.fc(out) 
    return out 

model = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
new_model = FeatureExtractor(model)

#### Get FC1 features of the VGG16 model

In [14]:
def concat_tensors(features):
    fc = torch.cat(features)
    fc = fc.cpu().detach().numpy()
    return fc

def get_features(tensors):
    with torch.no_grad():
        features = [new_model(tensor) for tensor in tensors]
    features = concat_tensors(features)
    return features

In [18]:
fc1 = get_features(image_tensors)
print(fc1.shape)

(291, 4096)


In [17]:
# save results
results = {'filename': files,
           'features': fc1,
           'layer_name': 'fc1'}

feature_dir = imgs_path.parent / 'features'
Path(feature_dir).mkdir(parents=True, exist_ok=True)
with open(feature_dir / 'VGG16_fc1_feature_std_224.pickle', 'wb') as f:
    pickle.dump(results, f)